In [1]:
import osmnx as ox;
import folium;
import pandas as pd;
from folium.plugins import FloatImage
import base64
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.point import Point

In [2]:
place = 'Bengaluru'

In [3]:
#Tags acquired from 'https://wiki.openstreetmap.org/wiki/Buildings'

community_centre = {'amenity':'community_centre'}
library = {'amenity':'library'}
community_building = ox.geometries_from_place(place, community_centre)
library_building = ox.geometries_from_place(place, library)

c:\Anaconda\envs\allgeo\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
c:\Anaconda\envs\allgeo\lib\site-packages\osmnx\utils_geo.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
c:\Anaconda\envs\allgeo\lib\site-packages\osmnx\utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
c:\Anaconda\envs\allgeo\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

In [4]:
community_building.head()

,unique_id,osmid,element_type,amenity,operator,geometry,name,ups_community,ups_school,addr:postcode,...,community_centre,boundary,addr:country,addr:hamlet,opening_hours,short_name,addr:suburb,email,community_centre:for,start_date
0,node/599205811,599205811,node,community_centre,Boy Scouts,POINT (77.57017 13.00306),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node/663604671,663604671,node,community_centre,NaN,POINT (77.57073 12.98498),S.C/S.T Welfare assocition,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node/663604681,663604681,node,community_centre,NaN,POINT (77.57077 12.98624),Dr.b R Ambedkar Samudaya Bavan(bbmp),community_centre,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node/663604682,663604682,node,community_centre,NaN,POINT (77.57075 12.98611),Youth welfare association,youth club,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node/663604697,663604697,node,community_centre,NaN,POINT (77.56819 12.99165),V.R.naidu Abimanigala sanga,NaN,school,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
community_building.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
print(len(community_building.index)) #All features in community_building = 166

community_building_points = community_building[community_building.geom_type == 'Point'][:100] #Only Point features in community_building = 85
print(len(community_building_points.index))

community_building_polygon = community_building[community_building.geom_type == 'Polygon'][:100] #Only Polygon features in community_building = 81
print(len(community_building_polygon.index))


166
85
81


In [7]:
community_building_points.head()

,unique_id,osmid,element_type,amenity,operator,geometry,name,ups_community,ups_school,addr:postcode,...,community_centre,boundary,addr:country,addr:hamlet,opening_hours,short_name,addr:suburb,email,community_centre:for,start_date
0,node/599205811,599205811,node,community_centre,Boy Scouts,POINT (77.57017 13.00306),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node/663604671,663604671,node,community_centre,NaN,POINT (77.57073 12.98498),S.C/S.T Welfare assocition,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node/663604681,663604681,node,community_centre,NaN,POINT (77.57077 12.98624),Dr.b R Ambedkar Samudaya Bavan(bbmp),community_centre,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node/663604682,663604682,node,community_centre,NaN,POINT (77.57075 12.98611),Youth welfare association,youth club,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node/663604697,663604697,node,community_centre,NaN,POINT (77.56819 12.99165),V.R.naidu Abimanigala sanga,NaN,school,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
locator = Nominatim(user_agent="google")


In [9]:
community_building_points["geocode"] = community_building_points['geometry'].y.map(str) + ',' + community_building_points['geometry'].x.map(str)
for ind in community_building_points.index:
    location = locator.reverse(community_building_points["geocode"][ind])
    community_building_points['geocode'][ind] = location.address
community_building_points

C:\Users\Pranav\AppData\Local\Temp\ipykernel_6160\2365976699.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  community_building_points['geocode'][ind] = location.address


,unique_id,osmid,element_type,amenity,operator,geometry,name,ups_community,ups_school,addr:postcode,...,boundary,addr:country,addr:hamlet,opening_hours,short_name,addr:suburb,email,community_centre:for,start_date,geocode
0,node/599205811,599205811,node,community_centre,Boy Scouts,POINT (77.57017 13.00306),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vidya Mandir School, West Park Road, Malleswar..."
1,node/663604671,663604671,node,community_centre,NaN,POINT (77.57073 12.98498),S.C/S.T Welfare assocition,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"S.C/S.T Welfare assocition, Platform Road, Rai..."
2,node/663604681,663604681,node,community_centre,NaN,POINT (77.57077 12.98624),Dr.b R Ambedkar Samudaya Bavan(bbmp),community_centre,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Dr.b R Ambedkar Samudaya Bavan(bbmp), Railway ..."
3,node/663604682,663604682,node,community_centre,NaN,POINT (77.57075 12.98611),Youth welfare association,youth club,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Youth welfare association, Railway Platform Ro..."
4,node/663604697,663604697,node,community_centre,NaN,POINT (77.56819 12.99165),V.R.naidu Abimanigala sanga,NaN,school,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"V.R.naidu Abimanigala sanga, 5th Main Road, Su..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,node/9632587538,9632587538,node,community_centre,NaN,POINT (77.56821 12.98286),RRR Kalyana Mantapa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"RRR Kalyana Mantapa, Kranthikavi Sarvagnya Roa..."
81,node/9796232684,9796232684,node,community_centre,NaN,POINT (77.61581 12.96975),Mother Tekla Auditorium,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Convent of St. Brigitta, 10, Brunton Road, Cra..."
82,node/9930197208,9930197208,node,community_centre,NaN,POINT (77.61221 12.96489),Shree Shwetambar Sthanakvasi Jain,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Shree Shwetambar Sthanakvasi Jain, 3rd Cross R..."
83,node/9938657069,9938657069,node,community_centre,NaN,POINT (77.58377 13.00697),All India Veerashiva Mahasabha,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"All India Veerashiva Mahasabha, Ramana Maharsh..."


In [10]:
community_building_polygon["centroid"] = community_building_polygon.to_crs(epsg=4326).centroid.to_crs(community_building_polygon.crs)
community_building_polygon["centroid"]

C:\Users\Pranav\AppData\Local\Temp\ipykernel_6160\3132305519.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  community_building_polygon["centroid"] = community_building_polygon.to_crs(epsg=4326).centroid.to_crs(community_building_polygon.crs)


85     POINT (77.65055 12.91784)
86     POINT (77.60004 12.89984)
87     POINT (77.60466 12.97043)
88     POINT (77.60286 12.93135)
89     POINT (77.60203 12.93136)
                 ...            
161    POINT (77.60250 12.99104)
162    POINT (77.55125 13.00581)
163    POINT (77.71090 12.98164)
164    POINT (77.60498 13.10962)
165    POINT (77.70732 12.91247)
Name: centroid, Length: 81, dtype: geometry

In [11]:
community_building_polygon["geocode"] = community_building_polygon['centroid'].y.map(str) + ',' + community_building_polygon['centroid'].x.map(str)
for ind in community_building_polygon.index:
    location = locator.reverse(community_building_polygon["geocode"][ind])
    community_building_polygon['geocode'][ind] = location.address
community_building_polygon = community_building_polygon.drop('centroid', axis=1)

C:\Users\Pranav\AppData\Local\Temp\ipykernel_6160\1310078364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  community_building_polygon['geocode'][ind] = location.address


,unique_id,osmid,element_type,amenity,operator,geometry,name,ups_community,ups_school,addr:postcode,...,addr:country,addr:hamlet,opening_hours,short_name,addr:suburb,email,community_centre:for,start_date,centroid,geocode
85,way/112692995,112692995,way,community_centre,NaN,"POLYGON ((77.65051 12.91787, 77.65059 12.91787...",HSR Residents Welfare Association,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.65055 12.91784),"HSR Residents Welfare Association, 10th Cross ..."
86,way/121004115,121004115,way,community_centre,NaN,"POLYGON ((77.60073 12.89995, 77.60077 12.89987...",Kalyani Kalamandir,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.60004 12.89984),"Kalyani Kalamandir, Bannerghatta Road, Arekere..."
87,way/124316744,124316744,way,community_centre,NaN,"POLYGON ((77.60474 12.97066, 77.60489 12.97035...",Good Shepherd Auditorium,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.60466 12.97043),"Good Shepherd Auditorium, Museum Road, Ashok N..."
88,way/131608822,131608822,way,community_centre,NaN,"POLYGON ((77.60278 12.93165, 77.60276 12.93162...",St. Thomas Parish Hall,NaN,NaN,560029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.60286 12.93135),"St. Thomas Parish Hall, Christ School Road, Sa..."
89,way/131608823,131608823,way,community_centre,NaN,"POLYGON ((77.60197 12.93151, 77.60192 12.93123...",Chandrodaya,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.60203 12.93136),"Chandrodaya, Christ School Road, Sadgunte Paly..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,way/1083566592,1083566592,way,community_centre,NaN,"POLYGON ((77.60217 12.99089, 77.60218 12.99124...",The Bangalore Friend-In Need Society,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,senior,1830,POINT (77.60250 12.99104),"The Bangalore Friend-In Need Society, Colonel ..."
162,way/1084356330,1084356330,way,community_centre,NaN,"POLYGON ((77.55143 13.00583, 77.55123 13.00590...",Vigneshwara Seva Samiti,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.55125 13.00581),"Vigneshwara Seva Samiti, 5th Cross Road, Rajaj..."
163,way/1084701604,1084701604,way,community_centre,NaN,"POLYGON ((77.71060 12.98184, 77.71073 12.98179...",Club House,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.71090 12.98164),"Club House, Seetharampalya Road, Green Domain ..."
164,way/1084735126,1084735126,way,community_centre,NaN,"POLYGON ((77.60461 13.11032, 77.60392 13.10921...",Missionaries of Charity,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (77.60498 13.10962),"Missionaries of Charity, Bellary Road, Yelahan..."


In [12]:
library_building.head()

,unique_id,osmid,element_type,addr:city,addr:housenumber,addr:postcode,addr:street,amenity,name,opening_hours,...,nodes,building,alt_name,building:colour,building:part,height,roof:colour,building:levels,is_in,operator:type
0,node/663568614,663568614,node,Bangalore,"23, Prestige Takt",560001,Kasturba Cross Road,library,British Council Library,Tu-Su 10:30-18:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node/674755223,674755223,node,NaN,NaN,NaN,NaN,library,RRI Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node/850751635,850751635,node,NaN,NaN,NaN,NaN,library,RT Nagar Public Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node/1018894416,1018894416,node,NaN,NaN,NaN,Elephant Cave Road,library,City Central Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node/1184210712,1184210712,node,NaN,NaN,NaN,NaN,library,KSCST Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
print(len(library_building.index)) #All features in community_building = 76

library_building_points = library_building[library_building.geom_type == 'Point'][:100] #Only Point features in community_building = 51
print(len(library_building_points.index))

library_building_polygon = library_building[library_building.geom_type == 'Polygon'][:100] #Only Polygon features in community_building = 25
print(len(library_building_polygon.index))

76
51
25


In [15]:
library_building_points["geocode"] = library_building_points['geometry'].y.map(str) + ',' + library_building_points['geometry'].x.map(str)
for ind in library_building_points.index:
    location = locator.reverse(library_building_points["geocode"][ind])
    library_building_points['geocode'][ind] = location.address
library_building_points

C:\Users\Pranav\AppData\Local\Temp\ipykernel_6160\2856389860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  library_building_points['geocode'][ind] = location.address


,unique_id,osmid,element_type,addr:city,addr:housenumber,addr:postcode,addr:street,amenity,name,opening_hours,...,building,alt_name,building:colour,building:part,height,roof:colour,building:levels,is_in,operator:type,geocode
0,node/663568614,663568614,node,Bangalore,"23, Prestige Takt",560001,Kasturba Cross Road,library,British Council Library,Tu-Su 10:30-18:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"British Council Library, 23, Prestige Takt, Ka..."
1,node/674755223,674755223,node,NaN,NaN,NaN,NaN,library,RRI Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"RRI Library, 2nd Cross Road, Sadhashivanagar, ..."
2,node/850751635,850751635,node,NaN,NaN,NaN,NaN,library,RT Nagar Public Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"RT Nagar Public Library, RT Nagar Main Road, G..."
3,node/1018894416,1018894416,node,NaN,NaN,NaN,Elephant Cave Road,library,City Central Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"City Central Library, Elephant Cave Road, Yedi..."
4,node/1184210712,1184210712,node,NaN,NaN,NaN,NaN,library,KSCST Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"KSCST Library, CV Raman Road, Sadhashivanagar,..."
5,node/1289827517,1289827517,node,NaN,NaN,NaN,NaN,library,City Central Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Malleshwaram 8th Cross Market, Sampige Road, M..."
6,node/1307456135,1307456135,node,NaN,NaN,NaN,NaN,library,Just books,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Just books, 27th Main Road, Sector 1, HSR Layo..."
7,node/1346450614,1346450614,node,NaN,NaN,NaN,NaN,library,Christ University Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Christ University Library, Dr. M H Marigowda R..."
8,node/1437223285,1437223285,node,NaN,NaN,NaN,NaN,library,Indian Medical Association,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Indian Medical Association, Albert Victor Road..."
9,node/1466921785,1466921785,node,Bangalore,NaN,560029,NaN,library,Christ University Library,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Christ University, 8th Cross Road, Sadgunte Pa..."


In [16]:
library_building_polygon["centroid"] = library_building_polygon.to_crs(epsg=4326).centroid.to_crs(library_building_polygon.crs)

C:\Users\Pranav\AppData\Local\Temp\ipykernel_6160\3452433460.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  library_building_polygon["centroid"] = library_building_polygon.to_crs(epsg=4326).centroid.to_crs(library_building_polygon.crs)


In [35]:
library_building_polygon["geocode"] = library_building_polygon['centroid'].y.map(str) + ',' + library_building_polygon['centroid'].x.map(str)
for ind in library_building_polygon.index:
    location = locator.reverse(library_building_polygon["geocode"][ind])
    library_building_polygon['geocode'][ind] = location.address
library_building_polygon = library_building_polygon.drop('centroid', axis=1)

C:\Users\Pranav\AppData\Local\Temp\ipykernel_6160\1474747916.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  library_building_polygon['geocode'][ind] = location.address


In [18]:
icon1 = folium.Icon(color='green',prefix='glyphicon',icon='home')
icon2 = folium.Icon(color='#FF7377 ',prefix='glyphicon',icon='book')

C:\Users\Pranav\AppData\Local\Temp\ipykernel_6160\3389431890.py:2: UserWarning: color argument of Icon should be one of: {'darkgreen', 'darkpurple', 'cadetblue', 'black', 'white', 'lightgray', 'lightblue', 'gray', 'pink', 'lightred', 'red', 'green', 'purple', 'darkblue', 'blue', 'lightgreen', 'orange', 'darkred', 'beige'}.
  icon2 = folium.Icon(color='#FF7377 ',prefix='glyphicon',icon='book')


In [36]:

m = folium.Map([12.9716, 77.5946], zoom_start=15, tiles='cartodbpositron', width=1800,height=1000)

folium.GeoJson(community_building_points,
               style_function=lambda x, mean_rsrp=community_building: {
                     'fillColor': 'yellow',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                marker =folium.Marker([45.5, -122.6], icon=icon1),
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)
folium.GeoJson(library_building_points,
               style_function=lambda x, mean_rsrp=library_building: {
                     'fillColor': 'blue',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                marker =folium.Marker([45.5, -122.6], icon=icon2),
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)
folium.GeoJson(community_building_polygon,
               style_function=lambda x, mean_rsrp=community_building: {
                     'fillColor': 'yellow',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)
folium.GeoJson(library_building_polygon,
               style_function=lambda x, mean_rsrp=library_building: {
                     'fillColor': 'blue',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                marker =folium.Marker([45.5, -122.6], icon=icon2),
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)

In [37]:
image_file = 'Legend.png'

FloatImage(image_file, left=80).add_to(m)

In [38]:
m

In [39]:
m.save('GeoBLR.html')